<a href="https://colab.research.google.com/github/tirthasheshpatel/TensorFlow2.0-Models/blob/master/TensorFlow_2_0_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.2
!git clone https://github.com/tirthasheshpatel/TensorFlow2.0-Models.git
%cd TensorFlow2.0-Models

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.2`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
fatal: destination path 'TensorFlow2.0-Models' already exists and is not an empty directory.
/content/TensorFlow2.0-Models


In [2]:
!git pull origin master

From https://github.com/tirthasheshpatel/TensorFlow2.0-Models
 * branch            master     -> FETCH_HEAD
Already up to date.


In [3]:
!pip install -r requirements.txt
!pip install -q tensorflow-hub
!pip install -q tfds-nightly

In [0]:
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from dataset import *
from models import *
from metrics import *
from train import *

%matplotlib inline

In [0]:
buffer_size   = 10000
batch_size    = 64
epochs        = 10
learning_rate = 0.0001

In [0]:
checkpoint_path = "training_1/cp.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [7]:
batched_train_data, batched_test_data, encoder = get_batched_imdb_dataset_from_tfds(buffer_size=buffer_size, batch_size=batch_size)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

history = model.fit(batched_train_data, epochs=epochs,
                    validation_data=batched_test_data, 
                    validation_steps=30,
                    callbacks=[cp_callback])

Epoch 1/10
391/391 [==============================] - ETA: 0s - loss: 0.6328 - accuracy: 0.5748
Epoch 00001: saving model to training_1/cp.ckpt
391/391 [==============================] - 45s 114ms/step - loss: 0.6328 - accuracy: 0.5748 - val_loss: 0.4415 - val_accuracy: 0.7974
Epoch 2/10
391/391 [==============================] - ETA: 0s - loss: 0.3447 - accuracy: 0.8518
Epoch 00002: saving model to training_1/cp.ckpt
391/391 [==============================] - 44s 112ms/step - loss: 0.3447 - accuracy: 0.8518 - val_loss: 0.3493 - val_accuracy: 0.8406
Epoch 3/10
391/391 [==============================] - ETA: 0s - loss: 0.2537 - accuracy: 0.9000
Epoch 00003: saving model to training_1/cp.ckpt
391/391 [==============================] - 44s 113ms/step - loss: 0.2537 - accuracy: 0.9000 - val_loss: 0.3253 - val_accuracy: 0.8693
Epoch 4/10
391/391 [==============================] - ETA: 0s - loss: 0.2140 - accuracy: 0.9192
Epoch 00004: saving model to training_1/cp.ckpt
391/391 [=============

In [0]:
myreview = "waste of time and money"
myreview = tf.convert_to_tensor([encoder.encode(myreview)])

In [9]:
model(myreview)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.95749325]], dtype=float32)>

In [0]:
myreview = "this movie was very amazing"
myreview = tf.convert_to_tensor([encoder.encode(myreview)])

In [11]:
model(myreview)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.69263387]], dtype=float32)>